# OraculumLocalBox for _pwsh_

There are two way to setup and experiment with a local installation of Oraculum and Sibylla. This interactive notebook is designed to explain how to use PowerShell to install and manage Oraculum and Sibylla on a local machine.

You can run these commands interactively in a PowerShell terminal without need for this notebook. However, this notebook is designed to be a step-by-step guide to setting up a local installation of Oraculum and Sibylla.

You need to install _Docker_ on your system and WSL2 if you are on Windows. You can find instructions for installing Docker on your system [here](https://docs.docker.com/get-docker/). You can find instructions for installing WSL2 on Windows [here](https://docs.microsoft.com/en-us/windows/wsl/install-win10).

## Keys

In this notebook we reference _oraculum.secret.conf_ and _weaviate.secret.yml_. Those are copies of _oraculum.conf_ and _weaviate.yml_ where you should indicate you OpenAI org id and key. You can get those from the [OpenAI dashboard](https://platform.openai.com/account/api-keys).

## First run

The initial deployment of _Oraculum_ requires the schema initialization first, then the Facts data structure can be populated. Follow the steps (in some cases a timeout may require re-evaluate the cell, or even restarting the kernels if stuck in evaluation):

In [2]:
# Start the _Weaviate_
docker-compose -f .\weaviate.secret.yml up -d

 Container weaviate  Running


You also need to install _Oraculum_ module from PowerShell Gallery. You can do that by running the following command in a PowerShell terminal:

In [3]:
Install-Module Oraculum

In [5]:
Import-Module Oraculum

Connect-Oraculum -ConfigFile .\oraculum.secret.conf


Configuration          IsConnected
-------------          -----------
Oraculum.Configuration        True



To initialize or reset the schema you use the `Reset-Schema` command. Notice that you will be prompted for entering an OTP to avoind unwanted exectution. Inside Visual Studio Code pay attention to the top of the window where an input box will be shown:

In [8]:
Reset-Schema


Schema reset
To reset the schema type in the code 384061
True


Now you can check that the fact class is empty:

In [9]:
Get-TotalFact

0


## Manage facts repository in _Oraculum_

Now that you have the system up and running you can start adding facts to the repository. You can do that by using the `Add-Fact` command. The command takes a single parameter, the fact to be added.

Remember to import the module and connect to _Oraculum_ first:

In [3]:
Import-Module Oraculum
Connect-Oraculum -ConfigFile .\oraculum.secret.conf


Configuration          IsConnected
-------------          -----------
Oraculum.Configuration        True




We first load a set of FAQs from a JSON file:

In [11]:
$json = Get-Content .\qamoviedb.json | ConvertFrom-Json
$json |% { Add-Fact -Category movie -FactType faq -Title $_.q -Content $_.a }

Note that we have added facts individually, this may take a while. The command `Add-Facts` would have been much faster but we should have created an array of `Fact` objects first. For small loads it is easier to add facts individually.

Now we can check that the facts have been added:

In [12]:
Get-TotalFact

287


You can use the `Get-Fact` command to retrieve a fact by its id, and the `Get-Facts` command to retrieve all facts in the repository. The `Get-Facts` command takes a single parameter, the number of facts to retrieve. If you do not specify a number, the limit is set to 1024:

In [14]:
Get-Facts -Limit 5


id         : 045733aa-d350-4b0e-b081-2e4f87ccbf03
distance   : 
factType   : faq
category   : movie
tags       : 
title      : Not counting the special achievement award won for the movie’s sound, how many 
             competitive Oscars did Star Wars Episode IV (A New Hope) win at the 50th Academy 
             Awards from its ten nominations?
content    : Six
citation   : 
reference  : 
expiration : 

id         : 0505d424-46e9-439c-a80f-1d25e8ae4e13
distance   : 
factType   : faq
category   : movie
tags       : 
title      : What is the name of the carnivorous creature from Tatooine that C-3PO is talking 
             about when he says "In its belly, you will find a new definition of pain and 
             suffering, as you are slowly digested over a thousand years?"
content    : Sarlacc
citation   : 
reference  : 
expiration : 

id         : 05f4535d-bfef-4942-a207-5e8ed9c3b893
distance   : 
factType   : faq
category   : movie
tags       : 
title      : What is the four-letter t

If you are running in a windows terminal you can use the `Out-GridView` command to display the facts in a separate window:

    Get-Facts | Out-GridView

_Oraculum_ uses _Weaviate_ ability to order facts by the embedding of the query (closer vectors are more relevant *even if the query is expressed in a different language than the one of the facts*). The Find-RelevantFacts command is used to see the distance between a query and the facts in the repository:

In [5]:
Find-RelevantFacts -Query "What do you know about Sarlacc?" -Limit 5 | select distance, title, content


distance title
-------- -----                                                                                     
   0.142 What is the name of the carnivorous creature from Tatooine that C-3PO is talking about wh…
   0.201 In the affectionate Star Wars parody, “Space Balls,” the film renames the slimy crime lor…
   0.202 Jabba The Hut’s pet/sidekick in "Return Of The Jedi", a noisy little rodent, is named Sal…
   0.209 George Lucas is on record saying that its shape was inspired by a hamburger, how is the l…
   0.211 What is the name of the large omnivorous cephalopods from the planet Vodran, having seven…



## Starting a Sibylla session

You can easily test your Sibylla session by using the `New-SibyllaSession` command. This command takes a single parameter, the configuration of the session. The command will interactively ask for prompts and will display answers until `#quit` will not be entered (again pay attention to the top of the Visual Studio Code window for the input box):

In [6]:
New-SibyllaSession -ConfigFile .\sibylla.conf

Assistant: Hello from your personal movie assistant. I know curiosities about 287 movies, try me!
You: Assistant: One carnivorous creature from a movie is the Sarlacc. It is a creature from Tatooine mentioned in Star Wars, and C-3PO describes it as a creature that slowly digests its victims over a thousand years [source: Star Wars: Episode VI - Return of the Jedi].
You: 